<a href="https://colab.research.google.com/github/inefable12/QuimicaComputacionalySimulaciones2025/blob/main/Clase3_C8987.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente material emplea RDKit y PySCF para la visualización de moléculas y para graficar los perfiles de energía, respectivamente. Sin embargo, estas no son las únicas herramientas, existen más herramientas complementarias para abordar estas temáticas, como por ejemplo:

1. Visualización de moléculas:
- Chimera
- VMD
- PyMOL
- Maestro Viewer

2. Perfil de energía
- MechaSVG https://github.com/ricalmang/mechaSVG
- EverPlot http://mor-vm-shiny.ethz.ch:3838/
- PyEnergyDiagrams: https://github.com/giacomomarchioro/PyEnergyDiagrams
- https://perfildeenergia.streamlit.app/

# A. Visualización de Moléculas usando RDKit

## A.1 Instalación de librerías

In [ ]:
%%capture
!pip install rdkit
!pip install pyscf

In [ ]:
# Importamos librerías
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
from pyscf import gto, scf

## A.2 Estructuras

### a. Isómero CIS

In [ ]:
# representación SMILES del isómero CIS
smi_cis = "F\C=C/C"
# elemento de RDKit
cis = Chem.MolFromSmiles(smi_cis)
# agregamos sus hidrógenos
cis = Chem.AddHs(cis)
# etiquetamos sus átomos y generamos sus coordenadas usando una semilla
Chem.AllChem.EmbedMolecule(cis, useRandomCoords=False,randomSeed=1234)
# guardamos a un archivo XYZ
Chem.MolToXYZFile(cis, "cis.xyz")

In [ ]:
#visualizamos
cis

In [ ]:
# visualizamos las coordenadas en el archivo XYZ
!cat cis.xyz

### b. Isómero TRANS

In [ ]:
smi_trans = "F/C=C/C"
trans = Chem.MolFromSmiles(smi_trans)
trans = Chem.AddHs(trans)
Chem.AllChem.EmbedMolecule(trans, useRandomCoords=False,randomSeed=1234)
Chem.MolToXYZFile(trans, "trans.xyz")

In [ ]:
trans

In [ ]:
!cat trans.xyz

### c. Diedro = 90.0 º

In [ ]:
Chem.AllChem.SetDihedralDeg(cis.GetConformer(0),3,2,1,0,90.0)
Chem.MolToXYZFile(cis, "diedro90.xyz")
cis

In [ ]:
!cat diedro90.xyz

## B. Cálculo de energía

### B.1 CIS

In [ ]:
cis = gto.M(atom="cis.xyz")

In [ ]:
cis.basis = "6-31G"
mf_cis = cis.KS()
mf_cis.xc = 'b3lyp'

In [ ]:
resultado_cis = mf_cis.kernel()
resultado_cis;

### B.2 Diedro 90

In [ ]:
cis_90 = gto.M(atom="diedro90.xyz")

In [ ]:
cis_90.basis = "6-31G"
mf_90 = cis_90.KS()
mf_90.xc = 'b3lyp'

In [ ]:
resultado_cis_90 = mf_90.kernel()
resultado_cis_90;

### B.3 TRANS

In [ ]:
trans = gto.M(atom="trans.xyz")

In [ ]:
trans.basis = "6-31G"
mf_trans = trans.KS()
mf_trans.xc = 'b3lyp'

In [ ]:
resultado_trans= mf_trans.kernel()
resultado_trans;

## C. Perfil de Energía

In [ ]:
%%capture
!pip install git+https://github.com/giacomomarchioro/PyEnergyDiagrams

In [ ]:
import matplotlib.pyplot as plt
from energydiagram import ED

In [ ]:
diagram = ED()
diagram.add_level(resultado_cis,'Cis',color='y')
diagram.add_level(resultado_cis_90,'TS',color='r')
diagram.add_level(resultado_trans,'Trans',color='g')

diagram.add_link(0,1)
diagram.add_link(1,2)
diagram.plot(ylabel="Energy") # / $kcal$ $mol^{-1}$")

### C.1 Cinética:

In [ ]:
E_activacion = resultado_cis_90 - resultado_cis
print(E_activacion)

### C.2 Termodinámica

In [ ]:
entalpia = resultado_trans - resultado_cis
if entalpia>0:
  print("La reacción es endotérmica")
else:
    print("La reacción es exotérmica")

## D. PyMOL

Ahora vamos a dirigirnos al programa PyMOL y emplearemos PDB RCSB para explorar diferentes representaciones para la visualización de proteínas, ligandos, etc.